In [2]:
# # Install Pytorch & other libraries, make sure to match your GPU driver version
# %pip install "torch==2.5.1" tensorboard "setuptools<71.0.0"  --index-url https://download.pytorch.org/whl/cu121

# # Install flash-attn
# %pip install flash-attn

# # Install Hugging Face libraries
# %pip install  --upgrade \
#   "transformers==4.48.1" \
#   "datasets==3.1.0" \
#   "accelerate==1.3.0" \
#   "hf-transfer==0.1.9" \
#   "deepspeed==0.15.4" \
#   "trl==0.14.0"

# # install vLLM
# %pip install "vllm==0.7.0"

# # ## IMPORTANT: If you want to run the notebook and the interactive cells you also need to install the following libraries:
# # # But first read it the blog post and then decide as they might conflict with the libraries for distributed training.
# # # %pip install "peft==0.14.0" "bitsandbytes==0.45.0"

In [3]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [4]:
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import LoraConfig
from trl import GRPOConfig, GRPOTrainer
import numpy as np

[2025-04-17 12:56:17,805] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3

INFO 04-17 12:56:19 __init__.py:183] Automatically detected platform cuda.


In [ ]:
# tensorboard --port 6007 --logdir /root/tf-logs/
# ps -ef | grep tensorboard | awk '{print $2}' | xargs kill -9 
# tensorboard --logdir=/root/autodl-tmp/outputs/Qwen-0.5B-GRPO/runs/Apr17_13-04-12_autodl-container-ef014e83a6-5e7d45d3 --port=6007
# tensorboard --logdir "C:\Users\Yuan\Desktop\YUAN\學業\北京清華\碩一下\LLM\Qwen-0.5B-GRPO\GRPO limited COT"
# autodl-tmp/outputs/Qwen-0.5B-GRPO/runs/Apr16_16-34-38_autodl-container-ef014e83a6-5e7d45d3

In [ ]:
# Load and prep dataset

SYSTEM_PROMPT = """
Respond in the following format:

<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip().replace(",", "").replace("$", "")

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': 'What is the largest single-digit prime number?'},
            {'role': 'assistant', 'content': XML_COT_FORMAT.format(
               reasoning="9 is divisble by 3 and 8 is divisible by 2, but 7 is prime.",
               answer="7"
            )},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# dataset = load_dataset(
#     'parquet',
#     data_files={
#         'train': 'gsm8k/train/train-00000-of-00001.parquet',
#         'test': 'gsm8k/test/test-00000-of-00001.parquet'
#     }
# )

# Reward functions

def correct_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [1.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def accuracy_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [1.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r, flags=re.DOTALL) for r in responses] 
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r, flags=re.DOTALL) for r in responses] 
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]


In [7]:
# # import torch
# from importlib.metadata import version

# # 检查 PyTorch 和 CUDA 版本
# print(f"PyTorch 版本: {torch.__version__}")
# print(f"CUDA 可用: {torch.cuda.is_available()}")
# print(f"CUDA 版本: {torch.version.cuda}")

# # 检查其他库版本
# libraries = [
#     "transformers", "datasets", "accelerate",
#     "deepspeed", "trl", "vllm", "peft", 
#     "bitsandbytes", "flash-attn"
# ]

# for lib in libraries:
#     try:
#         print(f"{lib} 版本: {version(lib)}")
#     except:
#         print(f"{lib} 未安装")

In [ ]:
from transformers import TrainerCallback, TrainerState, TrainerControl

class CustomSaveCallback(TrainerCallback):
    def on_step_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        if state.global_step >= 1100 and state.global_step % 100 == 0:
            control.should_save = True
        else:
            control.should_save = False
        return control

In [ ]:
#model_name = "meta-llama/Llama-3.2-1B-Instruct"

model_name = "models/Qwen2.5-0.5B-Instruct"

if "Llama" in model_name:
    output_dir = "outputs/Llama-1B-GRPO"
    run_name = "Llama-1B-GRPO-gsm8k"
else:
    output_dir="outputs/Qwen-0.5B-GRPO"
    run_name="Qwen-0.5B-GRPO-gsm8k"
    
training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=1e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=3,
    num_generations=6,
    max_prompt_length=256,
    max_completion_length=786,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    report_to="tensorboard",
    log_on_each_node=False,
)
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=64,
#     target_modules=["q_proj", "k_proj", "v_proj", "ao_proj", "up_proj", "down_proj", "gate_proj"],
#     task_type="CAUSAL_LM",
#     lora_dropout=0.05,
# )
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map=None
).to("cuda")
        
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# use peft at your own risk; not working for me with multi-GPU training
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        accuracy_func,
        correct_func],
    args=training_args,
    train_dataset=dataset,
    #peft_config=peft_config
)
trainer.train()

Current Batch Accuracy: 0.00
-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
The total number of whole points Emily has is 9*92=828 points. The total number of whole points Ahmed has is 91*9=819 points. Ahmed needs 828+9=937 total points to beat Emily. So if Ahmed wants to beat Emily, he needs to get at least 937-819=118 points on his final assignment. 
Extracted:
The total number of whole points Emily has is 9*92=828 points. The total number of whole points Ahmed has is 91*9=819 points. Ahmed needs 828+9=937 total points to beat Emily. So if Ahmed wants to beat Emily, he needs to get at least 937-819=118 points o

Step,Training Loss
1,0.000000
2,-0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


Current Batch Accuracy: 0.00
-------------------- Question:
In a graveyard, there are 20 skeletons.  Half of these skeletons are adult women, and the remaining number are split evenly between adult men and children.  If an adult woman has 20 bones in their body, and a male has 5 more than this, and a child has half as many as an adult woman, how many bones are in the graveyard? 
Answer:
375 
Response:
100
To determine the number of bones in the graveyard, we need to break down the information and calculate step by step.

1. **Identify the total number of skeletons and the breakdown:**
   - There are 20 skeletons in total.
   - Half of these skeletons are adult women. So, there are \(20 \times \frac{1}{2} = 10\) adult women.
   - The remaining skeletons are split evenly between adult men and children. So, there are \(20 - 10 = 10\) adult men and \(10\) children combined.

2. **Determine the bones each type of skeleton has:**
   - An adult woman has 20 bones.
   - A man has 5 more bones 